In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
from tensorflow.keras import layers

In [ ]:
inputs = tf.keras.Input(shape=(784, ), name='img')
h1 = layers.Dense(32, activation='relu')(inputs)
h2 = layers.Dense(32, activation='relu')(h1)
outputs = layers.Dense(10, activation='softmax')(h2)
model = tf.keras.Model(inputs=inputs, outputs=outputs, name='mnist model')

model.summary()
tf.keras.utils.plot_model(model, './images/mnist_model.png')
tf.keras.utils.plot_model(model, './images/model_info.png', show_shapes=True)

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784) / 255.0
x_test = x_test.reshape(10000, 784) / 255.0

In [ ]:
model.compile(optimizer=tf.optimizers.RMSprop(),
             loss=tf.losses.sparse_categorical_crossentropy,
             metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=5, batch_size=64)
test_scores = model.evaluate(x_test, y_test, verbose=0)
print('test loss:', test_scores[0])
print('test acc:', test_scores[1])
print(history.history)

In [ ]:
model.save('model1.h5')
del model
model = tf.keras.models.load_model('model1.h5')

In [ ]:
encoder_input = tf.keras.Input(shape=(28, 28, 1), name='img')
x = layers.Conv2D(16, 3, activation='relu')(encoder_input)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.Conv2D(16, 3, activation='relu')(x)
encoder_output = layers.GlobalMaxPooling2D()(x)

encoder = tf.keras.Model(encoder_input, encoder_output, name='encoder')
encoder.summary()

x = layers.Reshape((4, 4, 1))(encoder_output)
x = layers.Conv2DTranspose(16, 3, activation='relu')(x)
x = layers.Conv2DTranspose(32, 3, activation='relu')(x)
x = layers.UpSampling2D(3)(x)
x = layers.Conv2DTranspose(16, 3, activation='relu')(x)
decoder_output = layers.Conv2DTranspose(1, 3, activation='relu')(x)

autoencoder = tf.keras.Model(encoder_input, decoder_output, name='autoencoder')
autoencoder.summary()

In [ ]:
encoder_input = tf.keras.Input(shape=(28, 28, 1), name='original_img')
x = layers.Conv2D(16, 3, activation='relu')(encoder_input)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.Conv2D(16, 3, activation='relu')(x)
encoder_output = layers.GlobalMaxPooling2D()(x)

encoder = tf.keras.Model(encoder_input, encoder_output, name='encoder')
encoder.summary()

decoder_input = tf.keras.Input(shape=(16, ), name='encoded_img')
x = layers.Reshape((4, 4, 1))(decoder_input)
x = layers.Conv2DTranspose(16, 3, activation='relu')(x)
x = layers.Conv2DTranspose(32, 3, activation='relu')(x)
x = layers.UpSampling2D(3)(x)
x = layers.Conv2DTranspose(16, 3, activation='relu')(x)
decoder_output = layers.Conv2DTranspose(1, 3, activation='relu')(x)

decoder = tf.keras.Model(decoder_input, decoder_output, name='decoder')
decoder.summary()

autoencoder_input = tf.keras.Input(shape=(28, 28, 1), name='img')
encoded_img = encoder(autoencoder_input)
decoded_img = decoder(encoded_img)
autoencoder = tf.keras.Model(autoencoder_input, decoded_img, name='autoencoder')
autoencoder.summary()

In [ ]:
def get_model():
    inputs = tf.keras.Input(shape=(128, ))
    outputs = layers.Dense(1, activation='sigmoid')(inputs)
    return tf.keras.Model(inputs, outputs)

model1 = get_model()
model2 = get_model()
model3 = get_model()

inputs = tf.keras.Input(shape=(128, ))
y1 = model1(inputs)
y2 = model2(inputs)
y3 = model3(inputs)
outputs = layers.average([y1, y2, y3])
ensemble_model = tf.keras.Model(inputs, outputs)
keras.utils.plot_model(ensemble_model, './images/averange_model.png')

In [ ]:
num_tags = 12
num_words = 10000
num_departments = 4

title_input = keras.Input(shape=(None, ), name='title')
body_input = keras.Input(shape=(None, ), name='body')
tags_input = keras.Input(shape=(num_tags, ), name='tags')

title_features = layers.Embedding(num_words, 64)(title_input)
body_features = layers.Embedding(num_words, 64)(body_input)

title_features = layers.LSTM(128)(title_features)
body_features = layers.LSTM(32)(body_features)

x = layers.concatenate([title_features, body_features, tags_input])
priority_pred = layers.Dense(1, activation='sigmoid', name='priority')(x)
department_pred = layers.Dense(num_departments, activation='softmax', name='department')(x)

model = keras.Model(inputs=[title_input, body_input, tags_input],
                   outputs=[priority_pred, department_pred])
keras.utils.plot_model(model, './images/multi_input_and_output_model.png')

In [ ]:
model.compile(optimizer=keras.optimizers.RMSprop(1e-3),
             loss = ['binary_crossentropy', 'categorical_crossentropy'],
             loss_weights=[1., 0.2])
# model.compile(optimizer=keras.optimizers.RMSprop(1e-3),
#               loss={'priority': 'binary_crossentropy',
#                     'department': 'categorical_crossentropy'},
#               loss_weights=[1., 0.2])

In [ ]:
# Dummy input data
title_data = np.random.randint(num_words, size=(1280, 10))
body_data = np.random.randint(num_words, size=(1280, 100))
tags_data = np.random.randint(2, size=(1280, num_tags)).astype('float32')
# Dummy target data
priority_targets = np.random.random(size=(1280, 1))
dept_targets = np.random.randint(2, size=(1280, num_departments))

model.fit({'title': title_data, 'body': body_data, 'tags': tags_data},
          {'priority': priority_targets, 'department': dept_targets},
          epochs=2,
          batch_size=32)